In [1]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')

print("============================================")
print("/  MoteLoraDistance is running.            /")
print("============================================")

import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

import geopy
from geopy.distance import geodesic

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_SLACK_TOKEN = config['slack']['token']
_CHIRPSTACK_USER = config['chirpstack']['user']
_CHIRPSTACK_PASS = config['chirpstack']['password']
_DB_HOST  = config['kanjidb']['dbhost']
_DB_PORT  = config['kanjidb']['dbport']
_DB_NAME  = config['kanjidb']['dbname']
_DB_USER  = config['kanjidb']['dbuser']
_DB_PASS  = config['kanjidb']['dbpass']

_BATTERY_CRITICAL_VOLTAGE = float(config['analytics']['batterymin'])

_LOG_DEBUG = 0
_LOG_INFO  = 1
_LOG_ERROR = 2
_LOG_LEVEL = int(config['DEFAULT']['loglevel'])
_LOG_LEVEL = _LOG_INFO
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  logger(_LOG_INFO, "Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  logger(_LOG_ERROR, "You are not connected to the database.")

messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

/  MoteLoraDistance is running.            /
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [2]:
def postMessageToSlack(blockmessage):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = "rfsurvey"
    response = sc.api_call("chat.postMessage", channel=slackchannel, blocks=blockmessage)
    print(blockmessage) 
    if not 'ok' in response or not response['ok']:
      logger(_LOG_ERROR, "Error posting message to Slack channel")
      logger(_LOG_ERROR, blockmessage)
      logger(_LOG_ERROR, response)
    else:
      logger(_LOG_INFO, "Ok posting message to Slack channel")    

In [7]:
now = datetime.now(pytz.utc)  #tz Aware
logger(_LOG_DEBUG, "Current time is {}".format(now))

ticketnow = datetime.now(pytz.utc)  #tz Aware
now = datetime.now() + timedelta(hours = 4)

_NODE_STATUS_DEPLOYED = 10001
_NODE_TYPE_MOTE = 10005
_NODE_SENSORTYPE = 40           # the GEOLOCATION type

motequery = "SELECT * FROM kanji_node WHERE deploystate_id={} AND nodetype_id={}".format(_NODE_STATUS_DEPLOYED, _NODE_TYPE_MOTE)
logger(_LOG_DEBUG, motequery)
df = pd.read_sql(motequery, conn)

logger(_LOG_DEBUG, "number of motes {}".format(len(df.index)))
for ind in df.index:
    nodename = df['name'][ind]
    node_id = df['idnode'][ind]
    eventquery = "SELECT node.name AS gatewayname,\
                  * FROM kanji_eventlog event \
                  JOIN kanji_node node ON (node.name LIKE concat('%',event.gateway,'%')) \
                  JOIN kanji_location location ON location.idlocation=node.location_id \
                  JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
                  JOIN kanji_slackchannel slackchannel ON customer.slacksystemticketchannel_id=slackchannel.idslackchannel \
                  WHERE timestamp> NOW() - INTERVAL '1 MINUTE' AND node_id={} AND sensortype_id={} ORDER BY timestamp desc;".format(node_id, _NODE_SENSORTYPE)
    
    #logger(_LOG_INFO, eventquery)
    df2 = pd.read_sql(eventquery, conn)
    if len(df2.index)>0:
      _SLACK_TOKEN = df2["slacktoken"][0] 
      _SLACK_CHANNEL_NAME = df2["channelname"][0]
      _SLACK_CHANNEL_ID = df2["channelid"][0]
      _SLACK_CHANNEL_DBID = df2["idslackchannel"][0]  
      for ind2 in df2.index:
        timestamp = df2['timestamp'][ind2]
        latitude = df2['bval'][ind2]
        longitude = df2['cval'][ind2]
        altitude = df2['dval'][ind2]
        marcopolo = df2['eval'][ind2]
        lorasnr = df2['lorasnr'][ind2]
        rssi = df2['rssi'][ind2]
        gw_name = df2['gatewayname'][ind2]
        gw_latitude = df2['latitude'][ind2]
        gw_longitude = df2['longitude'][ind2]
        if ind2==0:
          logger(_LOG_INFO, "{}\t{}\tstar1\t{}\t{}".format(gw_latitude, gw_longitude, 'black', gw_name))
        #logger(_LOG_INFO,"\n{}".format(timestamp))
        #logger(_LOG_INFO, "Gateway {} {} {}".format(gw_name, gw_latitude, gw_longitude))
        #logger(_LOG_INFO, "Node {} {},{}".format(nodename, latitude, longitude))         
        nodelocation = (latitude, longitude)
        gatewaylocation = (gw_latitude, gw_longitude)
        loradistance = (float) (geodesic(nodelocation, gatewaylocation).miles * 5280/3)
        if rssi < -120.0:
            color = 'red'
        elif rssi < -100.0:
            color = 'yellow'
        else:
            color = 'green'
        if loradistance>50.0:
          logger(_LOG_INFO, "{}\t{}\tcircle1\t{}\t{}\tdist={:5.1f}yd snr={}dB, rssi={}dB".format(latitude, longitude, color, ind2, loradistance, lorasnr, rssi))
        geomessage = "{} *{} gateway reports:*\n*{} @ {:<5.1f}yards downrange.*\n*SNR={}dB, RSSI={}dB*" \
        .format(timestamp.strftime("%m/%d, %H:%M:%S"), gw_name, nodename, loradistance, lorasnr, rssi)
        #logger(_LOG_INFO, geomessage)
        blockmessage = json.loads(messagetemplate)
        # fixup message content
        blockmessage[0]["accessory"]["image_url"] = "https://www.dropbox.com/s/gft8wgriqjvma04/lora-snr.jpg?raw=1"
        blockmessage[0]["text"]["text"] = "{}".format(geomessage)
        #
        if marcopolo == 0.0:
         postMessageToSlack(blockmessage) 
logger(_LOG_INFO, "MoteLoRaDistance Done!") 

35.27448	-80.6483	star1	black	BullWinkle
[{'type': 'section', 'text': {'type': 'mrkdwn', 'text': '05/28, 19:59:01 *BullWinkle gateway reports:*\n*agMote-20002 @ 3.3  yards downrange.*\n*SNR=7.0dB, RSSI=-87dB*'}, 'accessory': {'type': 'image', 'image_url': 'https://www.dropbox.com/s/gft8wgriqjvma04/lora-snr.jpg?raw=1', 'alt_text': 'calendar thumbnail'}}]
Ok posting message to Slack channel
[{'type': 'section', 'text': {'type': 'mrkdwn', 'text': '05/28, 19:56:25 *BullWinkle gateway reports:*\n*agMote-20002 @ 9.4  yards downrange.*\n*SNR=7.0dB, RSSI=-87dB*'}, 'accessory': {'type': 'image', 'image_url': 'https://www.dropbox.com/s/gft8wgriqjvma04/lora-snr.jpg?raw=1', 'alt_text': 'calendar thumbnail'}}]
Ok posting message to Slack channel
[{'type': 'section', 'text': {'type': 'mrkdwn', 'text': '05/28, 19:55:12 *BullWinkle gateway reports:*\n*agMote-20002 @ 5.3  yards downrange.*\n*SNR=7.0dB, RSSI=-86dB*'}, 'accessory': {'type': 'image', 'image_url': 'https://www.dropbox.com/s/gft8wgriqjvma04